In [4]:
from helpers.config import vectorstore
#vectorstore.get()

c:\PythonWorkspace\server-pdf_handle\Server-RAG\helpers\config.py:12: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")
c:\PythonWorkspace\server-pdf_handle\Server-RAG\helpers\config.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [3]:
print(vectorstore.get().keys()) 

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'included'])


In [6]:
print(vectorstore.get()["metadatas"])

[{'author': 'MCC Support', 'creationDate': "D:20241228184304+00'00'", 'creator': 'Microsoft® Word 2016', 'end_page': 5, 'format': 'PDF 1.5', 'heading': 'Foreword', 'keywords': '<keyword[, keyword, ]>', 'modDate': 'D:20241228184304Z', 'producer': 'www.ilovepdf.com', 'source': '36201-i00.pdf', 'start_page': 4, 'subject': '<Title 1; Title 2> (Release 14 | 13 |12)', 'title': '3GPP TS ab.cde'}, {'author': 'MCC Support', 'creationDate': "D:20241226220500+00'00'", 'creator': 'Microsoft® Word 2016', 'end_page': 10, 'format': 'PDF 1.5', 'heading': '4.1 Overview', 'modDate': 'D:20241226220500Z', 'producer': 'www.ilovepdf.com', 'source': '38304-gb0.pdf', 'start_page': 9, 'subject': 'NR; User Equipment (UE) procedures in Idle mode and RRC Inactive state (Release 16)', 'title': '3GPP TS 38.304'}, {'author': 'MCC Support', 'creationDate': "D:20241226220500+00'00'", 'creator': 'Microsoft® Word 2016', 'end_page': 10, 'format': 'PDF 1.5', 'heading': '4.1 Overview', 'modDate': 'D:20241226220500Z', 'prod

In [20]:
print(vectorstore.get(where={"source": "36201-i00.pdf"})["ids"])

['bf89eaa1-2f9e-4d2d-9344-119021a7e6be']


In [ ]:
#tablo olarak görebileceğim kod
""" 
import pandas as pd
from IPython.display import display

# ChromaDB'den verileri al
vector_data = vectorstore.get()

# None olan anahtarları kontrol et
keys = ['ids', 'documents', 'metadatas']
valid_keys = [key for key in keys if vector_data.get(key) is not None]

# En küçük uzunluğu belirle (tüm sütunların eşit uzunlukta olması için)
min_length = min(len(vector_data[key]) for key in valid_keys)

# DataFrame oluştur
df = pd.DataFrame({
    'id': vector_data['ids'][:min_length],
    'document': vector_data['documents'][:min_length],
    'metadata': vector_data['metadatas'][:min_length]
})

# Tüm satır ve sütunların görüntülenmesini sağla
pd.set_option('display.max_rows', None)  # Tüm satırları göster
pd.set_option('display.max_columns', None)  # Tüm sütunları göster
pd.set_option('display.width', None)  # Çerçeveyi genişlet
pd.set_option('display.max_colwidth', None)  # Tüm hücrelerin içeriğini göster

display(df)
"""

" \nimport pandas as pd\nfrom IPython.display import display\n\n# ChromaDB'den verileri al\nvector_data = vectorstore.get()\n\n# None olan anahtarları kontrol et\nkeys = ['ids', 'documents', 'metadatas']\nvalid_keys = [key for key in keys if vector_data.get(key) is not None]\n\n# En küçük uzunluğu belirle (tüm sütunların eşit uzunlukta olması için)\nmin_length = min(len(vector_data[key]) for key in valid_keys)\n\n# DataFrame oluştur\ndf = pd.DataFrame({\n    'id': vector_data['ids'][:min_length],\n    'document': vector_data['documents'][:min_length],\n    'metadata': vector_data['metadatas'][:min_length]\n})\n\n# Tüm satır ve sütunların görüntülenmesini sağla\npd.set_option('display.max_rows', None)  # Tüm satırları göster\npd.set_option('display.max_columns', None)  # Tüm sütunları göster\npd.set_option('display.width', None)  # Çerçeveyi genişlet\npd.set_option('display.max_colwidth', None)  # Tüm hücrelerin içeriğini göster\n\ndisplay(df)\n"

In [ ]:
""" 
import os
import pymupdf
import gradio as gr
import shutil
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

def store_pdf(pdf_path, db_name="vector-db"):
    """
    #Process a PDF file, extract metadata, chunk it, and add to Chroma DB
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF file not found: {pdf_path}")
    
    pdf_filename = os.path.basename(pdf_path)
    
    try:
        pdf_open = pymupdf.open(pdf_path)
        toc = pdf_open.get_toc()
        pdf_metadata = pdf_open.metadata
        chunked_documents = []
        
        # If no table of contents, process entire document
        if not toc:
            full_text = "".join([page.get_text() for page in pdf_open])
            chunked_documents.append(
                Document(
                    page_content=full_text,
                    metadata={"source": pdf_filename, "heading": "Full Document"}
                )
            )
        else:
            for i, item in enumerate(toc):
                heading = item[1]
                start_page = item[2] - 1  # Adjust to 0-based indexing
                
                if i + 1 < len(toc):
                    end_page = toc[i + 1][2] - 1
                else:
                    end_page = pdf_open.page_count - 1
                
                chunk_text = "".join([pdf_open[page].get_text() for page in range(start_page, end_page + 1)])
                
                dynamic_metadata = {key: value for key, value in pdf_metadata.items() if value}
                
                chunked_documents.append(
                    Document(
                        page_content=chunk_text,
                        metadata={
                            "source": pdf_filename, 
                            "heading": heading, 
                            "start_page": start_page, 
                            "end_page": end_page, 
                            **dynamic_metadata
                        }
                    )
                )
        
        # Text splitting
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        final_chunks = text_splitter.split_documents(chunked_documents)
        
        # Print debug information
        print(f"Total chunks created: {len(final_chunks)}")
        
        # Embedding generation
        try:
            embeddings = OllamaEmbeddings(model="nomic-embed-text")
            
            # Test embedding generation
            test_embedding = embeddings.embed_documents([chunk.page_content for chunk in final_chunks[:1]])
            print(f"Embedding test: {len(test_embedding)} embeddings generated")
            
            # Chroma vector store
            vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
            
            # Add documents
            vectorstore.add_documents(final_chunks)
            
            print(f"Vectorstore updated with {len(final_chunks)} documents from {pdf_filename}")
            
            return f"PDF '{pdf_filename}' successfully processed and added to database."
        
        except Exception as embed_error:
            print(f"Embedding generation error: {embed_error}")
            return f"Error generating embeddings: {embed_error}"
    
    except Exception as process_error:
        print(f"PDF processing error: {process_error}")
        return f"Error processing PDF: {process_error}"
"""

In [ ]:
import os
import pymupdf
import gradio as gr
import shutil
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

def remove_pdf_from_db_1(pdf_name, db_name="vector-db"):
    """
    Remove all chunks associated with a given PDF file from the Chroma vector database.
    """
    try:        
        # Get all chunk IDs associated with the given PDF file
        chunk_ids = vectorstore.get(where={"source": pdf_name})["ids"]
        
        if not chunk_ids:
            print(f"No chunks found for PDF '{pdf_name}' in the vectorstore.")
            return f"No data found for PDF '{pdf_name}'."
        
        # Delete the chunks from the vectorstore
        vectorstore.delete(ids=chunk_ids)
        print(f"Successfully removed {len(chunk_ids)} chunks related to '{pdf_name}' from the vectorstore.")
        
        return f"PDF '{pdf_name}' successfully removed from the database."
    
    except Exception as e:
        print(f"Error removing PDF '{pdf_name}' from vectorstore: {e}")
        return f"Error: {e}"


In [10]:
import os
import shutil
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

def remove_pdf_from_db(pdf_name, db_name="vector-db", upload_folder="uploaded_files_admin"):
    """
    Remove all chunks associated with a given PDF file from the Chroma vector database
    and delete the PDF file from the uploaded files directory.
    
    Args:
        pdf_name (str): Name of the PDF file to remove
        db_name (str, optional): Name of the Chroma vector database directory. Defaults to "vector-db".
        upload_folder (str, optional): Directory containing uploaded PDF files. Defaults to "uploaded_files_admin".
    
    Returns:
        str: Status message of the removal process
    """
    try:
        # Load the existing Chroma vector store
        embeddings = OllamaEmbeddings(model="nomic-embed-text")
        vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
        
        # Get all chunk IDs associated with the given PDF file
        chunk_ids = vectorstore.get(where={"source": pdf_name})["ids"]
        
        if not chunk_ids:
            print(f"No chunks found for PDF '{pdf_name}' in the vectorstore.")
            """ 
            # Try to delete the file from the upload directory
            pdf_path = os.path.join(upload_folder, pdf_name)
            if os.path.exists(pdf_path):
                os.remove(pdf_path)
                print(f"Deleted PDF file '{pdf_name}' from {upload_folder}")
                return f"PDF '{pdf_name}' file deleted from storage."
            """
            return f"No data found for PDF '{pdf_name}'."
        
        # Delete the chunks from the vectorstore
        vectorstore.delete(ids=chunk_ids)
        print(f"Successfully removed {len(chunk_ids)} chunks related to '{pdf_name}' from the vectorstore.")
        
        # Delete the PDF file from the upload directory
        pdf_path = os.path.join(upload_folder, pdf_name)
        if os.path.exists(pdf_path):
            os.remove(pdf_path)
            print(f"Deleted PDF file '{pdf_name}' from {upload_folder}")
        
        return f"PDF '{pdf_name}' successfully removed from database and file storage."
    
    except Exception as e:
        print(f"Error removing PDF '{pdf_name}': {e}")
        return f"Error: {e}"

PDF YÜKLEME

In [8]:
from modules.addPDFtoDB import store_pdf
import os
import pymupdf
import gradio as gr
import shutil
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
# Gradio Interface
def save_and_process_pdf(pdf_file):
    """
    Save uploaded PDF and process it
    """
    if pdf_file is None:
        return "Please upload a PDF file."
    
    # Create upload folder if not exists
    UPLOAD_FOLDER = "uploaded_files_admin"
    os.makedirs(UPLOAD_FOLDER, exist_ok=True)
    
    # Save PDF
    pdf_filename = os.path.basename(pdf_file.name)
    save_path = os.path.join(UPLOAD_FOLDER, pdf_filename)
    shutil.copy2(pdf_file.name, save_path)

    # Process PDF
    result = store_pdf(save_path)

    return result

# Gradio Blocks Interface
with gr.Blocks() as demo:
    gr.Markdown("### Upload PDF and Add to Chroma DB")
    
    file_input = gr.File(label="Select a PDF")
    output_text = gr.Textbox(label="Result")

    submit_button = gr.Button("Upload and Process")

    submit_button.click(fn=save_and_process_pdf, inputs=file_input, outputs=output_text)

# Run Gradio interface
if __name__ == "__main__":
    demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Total chunks created: 21
Vectorstore updated with 21 documents from Unlocking_Telecom_Domain_Knowledge_Using_LLMs.pdf
Keyboard interruption in main thread... closing server.


In [9]:
print(vectorstore.get(where={"source": "Unlocking_Telecom_Domain_Knowledge_Using_LLMs.pdf"})["ids"])

['f6e1a24a-2719-4ca2-a7c4-654ccc7a65d0', '4d650a77-e095-4201-9cd6-62efacc5ab2c', 'a4aafd6a-254a-4f75-9f50-517b184de938', '614be069-6e1a-4c82-91bf-aff20c336c3f', 'efe6a1ae-8aa2-4f5e-b414-04ba4d5bb5e4', '92766173-cfc2-4f9a-8313-844cf749555f', '70bc76ca-87cc-4d1a-a009-7d7fe60e8df3', '940f6cf7-ed8f-4521-bb05-e6230e209067', '97d6f9ce-c6e3-42df-a175-da153570068c', '1341f3a5-1221-49bf-8cc2-79226c8b6eca', '88ccf1d3-93b0-4669-8a02-7a4b8e269a86', '75474d11-6d42-4cd5-b993-1596066189f2', '048bf9e6-5bd3-4134-b53a-f31b8d4f28a7', 'd1852caf-281d-4a9a-8cce-4649f5d71122', '7256764d-bad0-4d54-8255-62b865fa41ce', '140963e0-bdbf-4540-817f-cb8a6abd9874', 'ed240586-1d5c-4d9f-8ff4-33a8f7b62657', 'd4259149-c820-44e7-ab37-84b0d56754f6', '74d12d4b-1a37-4fc5-b29a-ea80043c09f0', '8a98d473-98f4-4695-9c56-0664d0f61dc1', '8719ec35-c00d-4b10-8810-2657ac271761']


gradio with remove pdf

In [ ]:
from modules.addPDFtoDB import store_pdf
import os
import pymupdf
import gradio as gr
import shutil
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

def save_and_process_pdf(pdf_file):
    """
    Save uploaded PDF and process it
    """
    if pdf_file is None:
        return "Please upload a PDF file."
    
    # Create upload folder if not exists
    UPLOAD_FOLDER = "uploaded_files_admin"
    os.makedirs(UPLOAD_FOLDER, exist_ok=True)
    
    # Save PDF
    pdf_filename = os.path.basename(pdf_file.name)
    save_path = os.path.join(UPLOAD_FOLDER, pdf_filename)
    shutil.copy2(pdf_file.name, save_path)

    # Process PDF
    result = store_pdf(save_path)

    return result

def list_uploaded_pdfs(upload_folder="uploaded_files_admin"):
    """
    List PDF files in the upload folder
    """
    try:
        pdfs = [f for f in os.listdir(upload_folder) if f.lower().endswith('.pdf')]
        return pdfs if pdfs else ["No PDFs uploaded"]
    except Exception as e:
        return [f"Error listing PDFs: {e}"]
    
# Gradio Blocks Interface
with gr.Blocks() as demo:
    gr.Markdown("### Upload PDF and Add to Chroma DB")
    
    file_input = gr.File(label="Select a PDF")
    output_text = gr.Textbox(label="Result")

    submit_button = gr.Button("Upload and Process")

    submit_button.click(fn=save_and_process_pdf, inputs=file_input, outputs=output_text)
    
    gr.Markdown("### Remove PDF from Database")
    
    # Refresh PDFs button and dropdown
    pdf_list = gr.Dropdown(label="Select PDF to Remove", choices=list_uploaded_pdfs())
    refresh_pdfs_btn = gr.Button("Refresh PDF List")
    refresh_pdfs_btn.click(fn=list_uploaded_pdfs, outputs=pdf_list)
        
        # Remove PDF button and output
    remove_output_text = gr.Textbox(label="Removal Result")
    remove_submit_button = gr.Button("Remove Selected PDF")
        
        # Remove PDF functionality
    remove_submit_button.click(
        fn=remove_pdf_from_db, 
        inputs=pdf_list, 
        outputs=remove_output_text
    )

# Run Gradio interface
if __name__ == "__main__":
    demo.launch(debug=True)    
""" 
# Gradio Blocks Interface
with gr.Blocks() as demo:
    gr.Markdown("## PDF Management System")
    
    with gr.Tab("Upload PDF"):
        gr.Markdown("### Upload PDF and Add to Chroma DB")
        file_input = gr.File(label="Select a PDF")
        upload_output_text = gr.Textbox(label="Upload Result")
        upload_submit_button = gr.Button("Upload and Process")
        upload_submit_button.click(fn=save_and_process_pdf, inputs=file_input, outputs=upload_output_text)

    with gr.Tab("Remove PDF"):
        gr.Markdown("### Remove PDF from Database")
        
        # Refresh PDFs button and dropdown
        pdf_list = gr.Dropdown(label="Select PDF to Remove", choices=list_uploaded_pdfs())
        refresh_pdfs_btn = gr.Button("Refresh PDF List")
        refresh_pdfs_btn.click(fn=list_uploaded_pdfs, outputs=pdf_list)
        
        # Remove PDF button and output
        remove_output_text = gr.Textbox(label="Removal Result")
        remove_submit_button = gr.Button("Remove Selected PDF")
        
        # Remove PDF functionality
        remove_submit_button.click(
            fn=remove_pdf_from_db, 
            inputs=pdf_list, 
            outputs=remove_output_text
        )

# Initial load of PDF list
demo.load(fn=list_uploaded_pdfs, outputs=pdf_list)

# Run Gradio interface
if __name__ == "__main__":
    demo.launch(debug=True)
    
"""

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\components\dropdown.py:188: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: ['35937-j00.pdf', 'Unlocking_Telecom_Domain_Knowledge_Using_LLMs.pdf'] or set allow_custom_value=True.
  warnings.warn(


Successfully removed 21 chunks related to 'Unlocking_Telecom_Domain_Knowledge_Using_LLMs.pdf' from the vectorstore.
Deleted PDF file 'Unlocking_Telecom_Domain_Knowledge_Using_LLMs.pdf' from uploaded_files_admin


In [ ]:
print(vectorstore.get(where={"source": "Unlocking_Telecom_Domain_Knowledge_Using_LLMs.pdf"})["ids"])